# **Importações e uploads necessários**

In [ ]:
import spacy
import re
import json
import pandas as pd

In [ ]:
!python -m spacy download pt_core_news_lg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_json('/content/drive/MyDrive/final_dataframe.json')
df_by_id_chat = df.set_index('_id_chat')
df_by_id_chat

In [ ]:
id_dialogos = df['_id_chat'].unique()
id_dialogos.size

1388

# **Funções extras**

In [ ]:
# validação do cpf

def valida_cpf(cpf_string):

    # retira apenas os dígitos do cpf

    numeros = [int(digito) for digito in cpf_string if digito.isdigit()]

    quant_digitos = False
    validacao1 = False
    validacao2 = False

    if len(numeros) == 11:
        quant_digitos = True

        # validação do primeiro dígito

        soma_produtos = sum(a*b for a, b in zip (numeros[0:9], range (10, 1, -1)))
        digito_esperado = (soma_produtos * 10 % 11) % 10
        if numeros[9] == digito_esperado:
            validacao1 = True

        # validação do segundo dígito

        soma_produtos1 = sum(a*b for a, b in zip(numeros [0:10], range (11, 1, -1)))
        digito_esperado1 = (soma_produtos1 *10 % 11) % 10
        if numeros[10] == digito_esperado1:
            validacao2 = True

        # validação geral

        if quant_digitos == True and validacao1 == True and validacao2 == True:
            return True
        else:
            return False

    else:
        return False



In [ ]:
# mapeamento estado do Ceará

ceara_state = {'Ceará', 'Abaiara', 'Acarape', 'Acarau', 'Acaraú', 'Acopiara', 'Aiuaba', 'Alencar', 'Altaneira', 'Antonina do Norte', 'Apuiarés',
               'Aquiraz', 'Aracati', 'Aracoiaba', 'Arapipaca', 'Araripe', 'Aratuba', 'Arneiroz', 'Assaré', 'Aurora', 'Baixio', 'Banabuiú', 'Barbalha',
               'Barreira', 'Barra do Ceará', 'Barro', 'Barroquinha', 'Baturité', 'Beberibe', 'Bela Cruz', 'Belém', 'Boa Viagem', 'Brejo Santo', 'Brisque',
               'Brasilia', 'Buriti', 'Campos Sales', 'Canindé', 'Capistrano', 'Caridade', 'Cariré', 'Caririaçu', 'Cariús', 'Carnaubal', 'Catarina', 'Catunda',
               'Cedro', 'Chaval', 'Chinopai', 'Choró', 'Chorozinho', 'Coreaú', 'Crateús', 'Crato', 'Croatá', 'Cruz', 'Deputado Irapuan Pinheiro', 'Ererê',
               'Eusebio', 'Farias Brito', 'Forquilha', 'Fortaleza', 'Frecheirinha', 'General Sampaio', 'Graça', 'Granja', 'Groaíras', 'Guaiúba', 'Guaraciaba do Norte',
               'Guaramiranga', 'Guaratuba', 'Hidrolândia', 'Ibaretama', 'Ibiapina', 'Icapuí', 'Icó', 'Iguatu', 'Independência', 'Ipaporanga', 'Ipu', 'Ipueiras',
               'Iracema', 'Irauçuba', 'Itaitinga', 'Itapagé', 'Itapipoca', 'Itapiúna', 'Itarema', 'Itatira', 'Itauá', 'Itauçú', 'Jardim', 'Jaguaribe', 'Jaguaruana',
               'Jangada', 'Jati', 'Jereissati', 'Jijoca de Jericoacoara', 'Juazeiro do Norte', 'Jucás', 'Jurema', 'Limoeiro do Norte', 'Mãe Luiza', 'Maracanaú',
               'Maranguape', 'Marco', 'Martinópole', 'Mauriti', 'Meruoca', 'Milagres', 'Milhã', 'Miraíma', 'Mombaça', 'Monsenhor Tabosa', 'Morada Nova', 'Morauju',
               'Mucambo', 'Mulungu', 'Nova Olinda', 'Nova Russas', 'Novo Oriente', 'Ocara', 'Orós', 'Pacajus', 'Pacatuba', 'Pacoti', 'Pacuja', 'Palhano', 'Paracuru',
               'Paraipaba', 'Parambu', 'Paramoti', 'Pedra Branca', 'Pedra de São Pedro', 'Pentecoste', 'Piquet Carneiro', 'Pires Ferreira', 'Poranga', 'Porteiras',
               'Potengi', 'Potiretama', 'Quixeramobim', 'Quixeré', 'Quiterianópolis', 'Redenção', 'Reriutaba', 'Russas', 'Saboeiro', 'Salitre', 'Santa Quitéria',
               'Santana do Acaraú', 'Santana do Cariri', 'Santo Antônio de Lisboa', 'São Benedito', 'São Gonçalo do Amarante', 'São João do Jaguaribe',
               'São João do Morros', 'São José de Ribamar', 'São Luís do Curu', 'São Mateus', 'São Miguel', 'São Pedro', 'São Tomé', 'São Valério do Sul',
               'Senador Pompeu', 'Senador Sá', 'Sobral', 'Solonópole', 'Tabuleiro do Norte', 'Tamboril', 'Tarrafas', 'Tauá', 'Tejuçuoca', 'Tianguá', 'Trairi',
               'Tururu', 'Umari', 'Umirim', 'Uruburetama', 'Uruoca', 'Varjota', 'Várzea Alegre', 'Viçosa do Ceará'}

# **Extração dos predicados do usuário através das entidades**

In [ ]:
def entities_extraction(phrase):

    nlp = spacy.load("pt_core_news_lg")

    # criação da regra da entidade e adição ao pipeline

    ruler = nlp.add_pipe("entity_ruler", before='ner')

    ruler.phrase_matcher = spacy.matcher.PhraseMatcher(nlp.vocab, attr="SHAPE")

    # entidades e seus respectivos padrões

    patterns = [
                    {
                        "label": "DOENCA", "pattern": [ {"LOWER": "coronavírus"} ], "id": "doenca"
                    },
                    {
                        "label": "DOENCA", "pattern": [ {"LOWER": "covid-19"} ], "id": "doenca"
                    },
                    {
                        "label": "ORG", "pattern": "Governo do Estado", "id": "organizacao"
                    },
                    {
                        "label": "TELEFONE", "pattern": [ {"TEXT": {"REGEX": "^\(?\d{2}\)?[-.\s]?\d{5}[-.\s]?\d{4}$"}} ], "id": "telefone"
                    },
                    {
                        "label": "TELEFONE", "pattern": '(49) 30492-2949', "id": "telefone"
                    },
                    {
                        "label": "CPF", "pattern": [ {"TEXT": {"REGEX": "(^(\d){11}$)"}} ], "id": "cpf"
                    },
                    {
                        "label": "CEP", "pattern": [ {"TEXT": {"REGEX": "(^(\d){8}$)"}} ], "id": "cep"
                    },
                    {
                        "label": "CEP", "pattern": [ {"TEXT": {"REGEX": "(^(\d){5}-(\d){3}$)"}} ], "id": "cep"
                    },
                    {
                        "label": "GENERO", "pattern": [ {"LOWER": "homem"} ], "id": "genero"
                    },
                    {
                        "label": "GENERO", "pattern": [ {"LOWER": "mulher"} ], "id": "genero"
                    },
                    {
                        "label": "DATA_NASC", "pattern": [ {"TEXT": {"REGEX": "^(0[1-9]|[12][0-9]|3[01])\/(0[1-9]|1[1,2])\/(19|20)\d{2}$"}} ], "id": "data_nascimento"
                    }
              ]

    # adição dos padrões as regras

    ruler.add_patterns(patterns)


    # criação do documento a partir do texto

    doc = nlp(phrase)

    # extração das entidades e suas respectivas labels

    find_labels = []

    for ent in doc.ents:
      if (ent.label_ != "MISC" and ent.label_ != "ORG" and ent.label_ != "DOENCA" and ent.text != "CPF" and ent.label_ != "LOC"):
        if (ent.label_ == "PER"):
          ent.label_ = "NOME"
        elif (ent.label_ == "CPF"):
          if not valida_cpf(ent.text):
            ent.label_ = "TELEFONE"
        #print (ent.text, ent.label_)
        find_labels.append(ent.label_)
      elif (ent.label_ == "LOC"):
        if (ent.text in ceara_state):
          #print (ent.text, ent.label_)
          find_labels.append(ent.label_)


    return find_labels


In [ ]:
def predicates():

    intentions_predicates = {'send-info-name': False, 'send-info-cpf': False, 'send-info-birthday': False, 'send-info-location': False,
                            'send-info-postal-code': False, 'send-info-gender': False, 'send-info-phone-number': False}
    entities_predicates = {'have-info-name': False, 'have-info-cpf': False, 'have-info-birthday': False, 'have-info-location': False,
                          'have-info-postal-code': False, 'have-info-gender': False, 'have-info-phone-number': False }

    return intentions_predicates, entities_predicates

In [ ]:
def predicates_extraction(user_phrase, int_predicates, ent_predicates):

    phrase = user_phrase

    list_predicates = entities_extraction(phrase)

    intentions_predicates = int_predicates
    entities_predicates = ent_predicates
    user_predicates = {}
    entity_found = ""

    if (len(list_predicates) == 0):
      return intentions_predicates, entities_predicates, user_predicates, entity_found
    else:
      for i in list_predicates:
        if (i == 'NOME'):
          entity_found = "Entidade NOME encontrada\n"
          intentions_predicates['send-info-name'] = True
          entities_predicates['have-info-name'] = True
        elif (i == 'CPF'):
          entity_found = "Entidade CPF encontrada\n"
          intentions_predicates['send-info-cpf'] = True
          entities_predicates['have-info-cpf'] = True
        elif (i == 'DATA_NASC'):
          entity_found = "Entidade DATA_NASC encontrada\n"
          intentions_predicates['send-info-birthday'] = True
          entities_predicates['have-info-birthday'] = True
        elif (i == 'LOC'):
          entity_found = "Entidade LOC encontrada\n"
          intentions_predicates['send-info-location'] = True
          entities_predicates['have-info-location'] = True
        elif (i == 'CEP'):
          entity_found = "Entidade CEP encontrada\n"
          intentions_predicates['send-info-postal-code'] = True
          entities_predicates['have-info-postal-code'] = True
        elif (i == 'GENERO'):
          entity_found = "Entidade GENERO encontrada\n"
          intentions_predicates['send-info-gender'] = True
          entities_predicates['have-info-gender'] = True
        elif (i == 'TELEFONE'):
          entity_found = "Entidade TELEFONE encontrada\n"
          intentions_predicates['send-info-phone-number'] = True
          entities_predicates['have-info-phone-number'] = True

      user_predicates.update(intentions_predicates)
      user_predicates.update(entities_predicates)

      return intentions_predicates, entities_predicates, user_predicates, entity_found

# **Análise dos diálogos do dataframe**

In [ ]:
# extração dos predicados do usuário a cada interação do usuário

file = open("/content/drive/MyDrive/user_predicates.txt","w")

dialogs_with_human = 0

for i in range(id_dialogos.size):
  initial_predicates = predicates()
  int_predicates = initial_predicates[0]
  ent_predicates = initial_predicates[1]
  dialog = df_by_id_chat.loc[id_dialogos[i]]
  if (isinstance(dialog,pd.DataFrame)):
    if dialog.astype(str).isin(['human']).any().any():
      dialogs_with_human += 1
    else:
      line = 'Predicados do diálogo ' + str(i) + '\n'
      file.writelines(line)
      line = 'Predicados iniciais do usuário: ' + str(initial_predicates) + '\n'
      file.writelines(line)
      for index, row in dialog.iterrows():
        if (row['ori'] == 'patient'):
          results = predicates_extraction(row.txt, int_predicates, ent_predicates)
          int_predicates = results[0]
          ent_predicates = results[1]
          file.write(str(results[3]))
          file.write(str(results[2]))
          file.write('\n')
      file.write('\n')

file.close()